In [1]:
# iMPORT PACKAGES
import pandas as pd
import datetime as dt
import numpy as np
import math
from datetime import timedelta
from statsmodels.tsa.vector_ar.var_model import VAR
from statsmodels.tsa.arima_model import ARIMA
import warnings
warnings.filterwarnings('ignore')


In [2]:
data = pd.read_csv("out_without_nan_mood_target.csv")
data['date'] = pd.to_datetime(data['date'])

In [3]:
df = data.dropna(subset = ['mood_mean','mood_mean_TARGET'] )

In [4]:
# Create test and train set
def createTrainTest(length_train):
    length_train = length_train
    index_test = np.random.choice(np.arange(len(df)), length_train, False)

    test = df.iloc[index_test]
    train = df.drop(df.index[index_test])
    
    return train, test

In [5]:
def calculateAccMSQ(solutions, name_target,name_predicted):
    correct = 0
    squared_error = 0

    for index, row in solutions.iterrows():
        squared_error += math.pow(row[name_target]-row[name_predicted],2)
        if (row[name_target] == row[name_predicted]):
            correct = correct + 1
        
    accuracy = correct/ len(solutions.index)
    msq = squared_error / len(solutions.index)
    return accuracy, msq


In [15]:
def predictMoodPersonARIMA(row, order):    
    #get train data
    train_data_person = df.loc[(df['id'] == row.id) & (df['date'] < row.date)]
    train_data_person_correct = train_data_person[["circumplex.valence_mean", 'mood_mean_TARGET']]
    
    prediction_mood_target = None
    #fit the model
    if len(train_data_person_correct) > 10:
        model = ARIMA(train_data_person_correct, order=order)
        model_fit = model.fit()
    
        # make prediction on validation
        prediction = model_fit.forecast()
        prediction_mood_target = prediction[0][0]
    
    return prediction_mood_target 

In [7]:
# Find for every row in test set the predicted mood
def predictARIMA(test, order):
    solutions = pd.DataFrame(data={'id': [], 'date': [], 'mood_mean_target': [], 'predicted_mood_mean_target':[]})
    for index, row in test.iterrows():
        prediction_mood_target = predictMoodPersonARIMA(row, order)
    
        #add solution to solution df
        if prediction_mood_target is not None:
            solutions = solutions.append({'id': row.id, 'date': row.date, 'mood_mean_target': row.mood_mean_TARGET,
                                      'predicted_mood_mean_target': prediction_mood_target}, ignore_index=True)
    return solutions

In [13]:
train, test = createTrainTest(100)

In [16]:
solutions = predictARIMA(test, (0,1,1))
accuracy, msq = calculateAccMSQ(solutions, "mood_mean_target", "predicted_mood_mean_target")
print(msq)

TypeError: must be str, not list

In [11]:
(1,0,0) = 0.23438219609414976
(0,1,0) = 0.43916991521125376
(0,2,0) = 1.591560337768915
(1,2,0) = 0.8812095702372237
(2,2,0) = 0.7547504645377935
(3,2,0) = error
(3,1,0) = 0.32514441411836276
(4,1,0) = error
(1,1,0) = 0.4145036325619722
(3,0,0) = 0.3425771089769928
(4,0,0) = 0.351796666069525
(0,0,1) = 0.3009717375417968
(0,0,2) = 0.3740895788190101
(0,0,3) = error
(1,0,1) = error
(0,1,1) = 0.24253054075758185
(1,1,1) = error
(0,2,1) = 0.5064616756505296

SyntaxError: can't assign to literal (<ipython-input-11-f4155a6eed94>, line 1)

In [14]:
for p in range(4):
    for d in range(4):
        for q in range(4):
            try:
                solutions = predictARIMA(test, (p,d,q))
                accuracy, msq = calculateAccMSQ(solutions, "mood_mean_target", "predicted_mood_mean_target")
                x1= p,d,q
                print (x1,msq)
            except:
                pass

(0, 0, 0) 0.5963085525488421
(0, 0, 1) 0.5925251086009311
(0, 1, 0) 0.7238104534814574
(0, 1, 1) 0.7064291839253898
(0, 2, 0) 2.0008499237760695
(0, 2, 1) 0.8353811450760794
(1, 0, 0) 0.5392210710514569
(1, 1, 0) 0.5275432630615116
(1, 2, 0) 1.0012711784712498
(2, 0, 0) 0.5228845034698109
(2, 1, 0) 0.6216914720842214
(2, 2, 0) 0.8748528406610534
(3, 0, 0) 0.529722920838077
(3, 1, 0) 0.6468895646711179
